# CNN 실습

앞서 살펴본 예제를 바탕으로, 실제로 코드를 작성하는 실습을 진행합니다.

In [ ]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

## Rock-Paper-Scissors 데이터셋 로드

Rock-Paper-Scissors 데이터셋을 로드합니다.

Rock-Paper-Scissors 데이터셋은 가위, 바위, 보 게임을 하는 손 이미지를 모은 데이터셋으로, 300x300x3 크기의 이미지들로 구성되어 있습니다.

In [ ]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, 'rps.zip')
local_zip = 'rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/')
zip_ref.close()

## 로드한 데이터 확인

예시로 몇 개의 데이터만 확인해보겠습니다.

현재 경로의 tmp/rps 폴더 안에 paper, rock, scissors 폴더와 이미지가 생성된 것을 확인할 수 있습니다.

Rock-Scissors-Paper Dataset의 세부적인 정보는 아래 링크에서 확인할 수 있습니다.

https://www.tensorflow.org/datasets/catalog/rock_paper_scissors

In [ ]:
!ls tmp/rps/paper/* | head -20

## **MISSION: 데이터 전처리**
## **예시 답안**
이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

TFDS 패키지를 이용한 것이 아니므로 ImageDataGenerator를 이용하여 전처리를 진행할 것입니다.

<br>

0-255 사이의 pixel 값을 0-1 사이의 float 값으로 Normalize를 실시하고,

Train / Validation 의 비율을 8:2로 설정하겠습니다.

<br>

설정한 데이터를 flow_from_directory를 이용하여 Resize, batch_size, class_mode를 지정하겠습니다.

데이터의 크기는 150x150 으로 Resize를 실시할 것이며,
batch_size는 20,
class_mode는 categorical로 지정하겠습니다.

여기서 categorical로 지정하면 label 정보가 one-hot encoding으로 설정됩니다.

<br>

ImageDataGenerator와 flow_from_directory에 관련한 상세한 자료는 아래 TensorFlow Document에서 확인하실 수 있습니다.

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory


In [ ]:
TRAINING_DIR = "tmp/rps/"

training_datagen = ImageDataGenerator(
  rescale=1. / 255,
  validation_split=0.2
)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                       target_size=(150, 150), 
                                                       batch_size=20, 
                                                       class_mode='categorical', 
                                                       subset='training',
                                                      )

validation_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                            target_size=(150, 150), 
                                                            batch_size=20, 
                                                            class_mode='categorical',
                                                            subset='validation',
                                                           )

## **MISSION: 네트워크 정의**
## **예시 답안**

먼저, Conv2D와 MaxPooling2D를 번갈아가며 추가하는 것으로 Feature Extractor를 구성하도록 합니다.

2회 혹은 3회 가량 Conv2D와 MaxPooling2D를 연결한 다음, Classifier를 구성하기 위해 Flatten 레이어를 추가합니다.

Flatten 레이어 이후에는 Dense 레이어를 통해 Fully Connected 레이어 계산을 해주도록 합니다.

<br>

그리고 마지막에 Rock-Scissors-Paper의 세 개의 class 분류를 위해서 3개의 출력 값을 갖는 Dense 레이어를 배치하도록 합니다.

마지막 Dense 레이어의 activation 함수로는 softmax를 지정합니다.

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

## **MISSION: 네트워크 학습**
## **예시 답안**

아까 flow_from_directory에서 Categorical로 class_mode를 지정하였으니 loss 함수로는 Catogoricalentropy를 사용합니다.

혹시나 sparse로 지정하였다면, SparseCategoricalCrossentropy를 사용하여 compile 합니다.

Optimizer의 경우에는 SGD, Adam, RMSprop, ... 등 아무거나 선택하셔도 무방합니다.

<br>

학습은 15 epoch을 실시하였고, 더 줄이거나 늘려도 문제 없습니다.

validation accuracy가 80% 이상 나오도록 위의 네트워크 구조 및 학습 epoch 수를 변경하시면 좋습니다.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_generator, epochs=15, validation_data=validation_generator)

## 참고 사항

학습 중 Train Accuracy가 100%가 되고, 점차 Train loss가 늘어나는 것으로 보아

과대적합(Overfitting)이 발생함을 알 수 있습니다.

이를 해결하고, 성능 향상을 위하여 ImageDataGenerator에 Augmentation을 적용할 수 있습니다.

다만, Augmentation 적용 시 학습 속도가 느려질 수 있습니다.

해당 코드는 아래를 참고하시면 됩니다.

In [ ]:
training_datagen = ImageDataGenerator(
  rescale=1. / 255,
  validation_split=0.2,
  rotation_range=5,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True
)

train_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                       target_size=(150, 150), 
                                                       batch_size=20, 
                                                       class_mode='categorical', 
                                                       subset='training',
                                                      )

validation_generator = training_datagen.flow_from_directory(TRAINING_DIR, 
                                                            target_size=(150, 150), 
                                                            batch_size=20, 
                                                            class_mode='categorical',
                                                            subset='validation',
                                                           )